In [1]:
!pip3 install torch torchvision

In [0]:
import torch
import torch.nn as nn

### Data

In [0]:
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) # 3 X 2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) # 3 X 1 tensor
xPredicted = torch.tensor(([4, 8]), dtype=torch.float) # 1 X 2 tensor

In [4]:
print(X.size())
print(y.size())

torch.Size([3, 2])
torch.Size([3, 1])


### Scaling

对数据做归一化的操作

In [5]:
# scale units
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(xPredicted, 0)

X = torch.div(X, X_max)
xPredicted = torch.div(xPredicted, xPredicted_max)
y = y / 100  # max test score is 100
print(xPredicted)
print(X)
print(y)

tensor([0.5000, 1.0000])
tensor([[0.6667, 1.0000],
        [0.3333, 0.5556],
        [1.0000, 0.6667]])
tensor([[0.9200],
        [1.0000],
        [0.8900]])


### Model

建立网络模型

In [0]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 2 X 3 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor

        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))
        

In [7]:
NN = Neural_Network()
for i in range(1000):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

#0 Loss: 0.06439349055290222
#1 Loss: 0.04925112798810005
#2 Loss: 0.039051517844200134
#3 Loss: 0.03190300241112709
#4 Loss: 0.026714110746979713
#5 Loss: 0.022832700982689857
#6 Loss: 0.019854051992297173
#7 Loss: 0.017517797648906708
#8 Loss: 0.015650788322091103
#9 Loss: 0.014134515076875687
#10 Loss: 0.01288565993309021
#11 Loss: 0.011844388209283352
#12 Loss: 0.010966721922159195
#13 Loss: 0.010219814255833626
#14 Loss: 0.009578733704984188
#15 Loss: 0.009024213999509811
#16 Loss: 0.008541240356862545
#17 Loss: 0.008117935620248318
#18 Loss: 0.007744795177131891
#19 Loss: 0.007414163555949926
#20 Loss: 0.00711977481842041
#21 Loss: 0.006856521591544151
#22 Loss: 0.006620144471526146
#23 Loss: 0.006407087203115225
#24 Loss: 0.006214401219040155
#25 Loss: 0.006039546802639961
#26 Loss: 0.005880414042621851
#27 Loss: 0.005735166370868683
#28 Loss: 0.0056022400967776775
#29 Loss: 0.005480291787534952
#30 Loss: 0.005368145648390055
#31 Loss: 0.005264792125672102
#32 Loss: 0.0051693418

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Neural_Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
